In [1]:
# Instalar bibliotecas necessárias
!pip install tensorflow matplotlib kaggle

In [ ]:
# Importar bibliotecas
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import zipfile
import os


In [ ]:
# Carregar a chave da API do Kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Baixar o dataset do Kaggle
!kaggle datasets download -d vencerlanz09/sea-animals-image-dataste

# Descompactar o dataset
with zipfile.ZipFile('sea-animals-image-dataste.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset')

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/vencerlanz09/sea-animals-image-dataste
License(s): other
sea-animals-image-dataste.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# Configurar os geradores de dados

INPUT_SHAPE = (224,224,3)
BATCH_SIZE = 32
EPOCHS = 5

datagen = ImageDataGenerator(
            rotation_range=20,
            horizontal_flip=True,
            vertical_flip=True,
            rescale=1./255,
            validation_split=0.2
                )

train_generator = train_datagen.flow_from_directory(
    "/content/dataset",  # Caminho para o diretório de treino
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    "/content/dataset",  # Caminho para o diretório de validação
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)


Found 13711 images belonging to 23 classes.
Found 13711 images belonging to 23 classes.


In [ ]:
# Construir o modelo
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinar o modelo
history = model.fit(
    train_generator,
    steps_per_epoch=100,  # Número de batches por epoch
    epochs=5,
    validation_data=validation_generator,
    validation_steps=50  # Número de batches de validação por epoch
)

# Salvar o modelo treinado
model.save('marine_life_model.h5')

Epoch 1/5
100/100 [==============================] - 141s 1s/step - loss: -2809639936.0000 - accuracy: 0.0301 - val_loss: -21680320512.0000 - val_accuracy: 0.0350
Epoch 2/5
100/100 [==============================] - 142s 1s/step - loss: -552766865408.0000 - accuracy: 0.0310 - val_loss: -2144953958400.0000 - val_accuracy: 0.0380
Epoch 3/5
100/100 [==============================] - 142s 1s/step - loss: -10634758455296.0000 - accuracy: 0.0340 - val_loss: -25999739715584.0000 - val_accuracy: 0.0440
Epoch 4/5
100/100 [==============================] - 139s 1s/step - loss: -70431943950336.0000 - accuracy: 0.0360 - val_loss: -131905525645312.0000 - val_accuracy: 0.0350
Epoch 5/5
100/100 [==============================] - 134s 1s/step - loss: -286557265199104.0000 - accuracy: 0.0345 - val_loss: -494735470886912.0000 - val_accuracy: 0.0220


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
def predict_image(image_path):
    # Carregar o modelo treinado
    model = tf.keras.models.load_model('/content/marine_life_model.h5')

    # Carregar e processar a imagem
    img = load_img(image_path, target_size=(150, 150))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    # Fazer a previsão
    prediction = model.predict(img_array)
    if prediction[0] > 0.5:
        print("A espécie está em extinção.")
    else:
        print("A espécie não está em extinção.")

    # Mostrar a imagem
    plt.imshow(img)
    plt.axis('off')
    plt.show()

# Usar a função de previsão
image_path = '/content/dataset/Fish/110666698_13f5024d23_o.jpg'  # Substituir pelo caminho da imagem a ser verificada
predict_image(image_path)

In [ ]:
# Visualizar a acurácia e a perda durante o treinamento
plt.figure(figsize=(12, 4))


#O objetivo é observar como a acurácia do modelol, isso ajuda a identificar se o modelo está aprendendo bem.
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Acurácia de Treinamento')
plt.plot(history.history['val_accuracy'], label='Acurácia de Validação')
plt.legend()
plt.title('Acurácia de Treinamento e Validação')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Perda de Treinamento')
plt.plot(history.history['val_loss'], label='Perda de Validação')
plt.legend()
plt.title('Perda de Treinamento e Validação')

plt.show()